In [1]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Concatenate
from keras.regularizers import l1_l2
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras import backend as K
from sklearn.cross_validation import StratifiedKFold
import csv

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Here, we clean up the data. We create 2 dataframes, one for winners and one for losers. Each has the same column names (team, score, fgm, fga, fgm3, fga3, ftm, fta, or, dr, ast, stl, blk, pf).
<br> We will now use these dataframes to build our representative team vectors for the season, putting together the information from the winners and the losers.

In [2]:
all_attributes = ['team', 'score', 'fgm', 'fga', 'fgm3', 'fga3', 'ftm', 'fta', 'or', 'dr', 'ast', 'stl', 'blk', 'pf']
imp_attributes = ['team', 'score', 'fgm', 'fga', 'fgm3', 'fga3', 'ftm', 'fta', 'or', 'dr', 'ast', 'stl', 'blk']

def load_team_data():
    reg_season_data_filename = "RegularSeasonDetailedResults.csv"
    #We load the data into a pandas dataframe.
    reg_season_data = pd.read_csv(reg_season_data_filename)
    # reg_season_data = reg_season_data.loc[lambda df: df.Season == 2003]
    #Took out location and overtimes. Play around with what other stats we want to include. 
    
    w = reg_season_data[['Season'] + ['W' + x for x in imp_attributes]]
    win_remap = {'W'+x: x for x in imp_attributes}
    winners = w.rename(index = str, columns = win_remap)
    los_remap = {'L' + x:x for x in imp_attributes}
    l = reg_season_data[['Season']+ ['L' + x for x in imp_attributes]]
    losers = l.rename(index=str, columns = los_remap)
    
    #Know what seasons of data we are dealing with
    reg_season_years = reg_season_data['Season'].unique()

    #Here, we put all the data (from both winners and losers) into one dataframe. 
    team_data = pd.concat([winners, losers])

    by_reg_season = {}
    for year in reg_season_years:
        s = team_data[team_data['Season']==year]
    #     display(s)
        s = s.drop(['Season'], axis = 1)
        by_reg_season[year] = s
    return by_reg_season, reg_season_years


In [3]:
by_reg_season, reg_season_years = load_team_data()
# display(by_reg_season)

We denominate the team vector to be represented as:
\[Score, FG Made, FG Attempted, 3 Pointers Made, 3 Pointers attempted, FT made, FT Attempted, Off Rebounds, Def Rebounds, Assists, Steals, Blocks\]. (Later in the code we add the team's seed to their vectors)
<br> <br>
In this following cell we aggregate the data and calculate each team's average statistics for each year. To store this, we keep 1 dataframe (indexed by team) for each year of data. The pointers to these objects are stored in our dictionary, which serves as a database.

In [4]:
def get_team_vector_store(by_reg_season, reg_season_years):
    # Create dictionary mapping year-> Dataframe of teams within that year.
    team_vector_store = {}

    for year in reg_season_years:
        t = by_reg_season[year].groupby('team')[imp_attributes[1:]].mean()
        team_vector_store[year] = t.apply(lambda x: x, axis = 0)
    
    return team_vector_store


In [5]:
team_vector_store = get_team_vector_store(by_reg_season, reg_season_years)

We define a function to normalize the data. This could be used at whatever point we want. If we want a standardn normal distribution, set standard = True, otherwise we let standard=False for min/max normalization.

We test using both methods of normalization to see which gave us better results.

In [6]:
def normalize_data(team_vector_store, standard = False):
    """
    Function to normalize the data. 
    IMPORTANT: Modifies the argument passed in.
    Set standard to True if normalizing to standard Gaussian,
    False for min/max normalization. 
    """
    for y in reg_season_years:
        if standard:
            team_vector_store[y] = team_vector_store[y].apply(lambda x: (x - x.mean())/x.std(), axis = 0)
        else:
            team_vector_store[y] = team_vector_store[y].apply(lambda x: x/x.max(), axis = 0)

We define a matchup to be the difference between the vectors of team1 and team2. If team 1 wins, we classify the matchup as a 1, while we classify it as a 0 if team2 wins. Below is a quick example of how it could work. However, we must now add each team's seed to their representative vector.

In [7]:
year = 2003
m = team_vector_store[year]
matchup_1 = (m.loc[1102] - m.loc[1103])
matchup_1['class'] = 0
# display(matchup_1)

Below, we modify each season's team vector dataframe by adding each team's NCAA tournament seed. This will allow us to use this as a feature when outputting predictions.

In [8]:
def load_seed_data(filename, standardize = False):
    tourney_seeds_data = pd.read_csv(filename)
    tourney_seeds_data = tourney_seeds_data.query('Season >= 2003')
    tourney_seeds_data = tourney_seeds_data.reset_index(drop=True)
    tourney_seeds_data.rename(str.lower, axis='columns', inplace = True)
    tourney_seeds_data['seed'] = tourney_seeds_data['seed'].str.extract('(\d+)', expand = False).astype(int)
    if standardize:
        tourney_seeds_data['seed'] = tourney_seeds_data['seed']/16
    return tourney_seeds_data

In [9]:
tourney_seeds_data = load_seed_data("TourneySeeds.csv")

In [10]:
def update_vector_store(team_vector_store, tourney_seeds_data):
    #Joins the team vector store with the seed information.
    #Allows us to consider seed as an attribute of the team.
    #Eliminates the teams not in the tournament.
    for s in reg_season_years:
        relevant_data = tourney_seeds_data.query('season == {}'.format(s))
        relevant_data = relevant_data.drop('season', axis = 1)
        m = team_vector_store[s]
        m = m.merge(relevant_data, how = 'outer', left_index = True, right_on = 'team')
        m.set_index('team', inplace=True)
        m = m[pd.notnull(m['seed'])]
        team_vector_store[s] = m
    return team_vector_store

In [11]:
team_vector_store = update_vector_store(team_vector_store, tourney_seeds_data)

In [12]:
normalize_data(team_vector_store, standard = True)

Now, we must use our team vectors along with the matchups each season to build our feature vectors, which we will use for classification. A 1 will signify that the 1st team won, while a 0 means the 2nd team won.

Our NCAA tournament data is in the same format as the regular season one. In order to have a combination of 1s and 0s, we will generate 2 different vectors for each matchup, where the first has team_1 = winning_team while the second has team_1 = losing_team. Because of the way our data is labeled, if we do not do this then all of our vectors will have classification 1 or 0, which won't allow our model to actually learn a decision boundary.

In [13]:
def generate_training_data(team_vector_store, classification = True):
    """
    Function to generate the training/testing data.
    Based on team_vector_store and the tourney detailed results.
    If classification = True, generates for binary classification.
    If classification = False, generates for regression.
    
    Returns training_data
    """
    tourney_game_data = pd.read_csv("TourneyDetailedResults.csv")
    tourney_game_data = tourney_game_data[['Season', 'Wteam', 'Lteam', 'Wscore', 'Lscore']]


    col_names = list(team_vector_store[2003]) + ['class']

    train_matchup_data = pd.DataFrame(columns = col_names)

    for index, row in tourney_game_data.iterrows():
        season, wteam, lteam, wscore, lscore = row
        
        #Winner is team 1, loser is team 2
        game_vector_1 = team_vector_store[season].loc[wteam] - team_vector_store[season].loc[lteam]
        if classification:
            game_vector_1['class'] = 1
        else:
            game_vector_1['class'] = wscore - lscore
        
        #Loser is team 1, winner is team 2
        game_vector_2 = team_vector_store[season].loc[lteam] - team_vector_store[season].loc[wteam]
        if classification:
            game_vector_2['class'] = 0
        else:
            game_vector_2['class'] = lscore - wscore
        
        train_matchup_data = train_matchup_data.append([game_vector_1, game_vector_2], ignore_index = True)
        
    return train_matchup_data

In [14]:
train_matchup_data_class = generate_training_data(team_vector_store, classification = True)
train_matchup_data_reg = generate_training_data(team_vector_store, classification = False)

We now have our training data built up. The X data will just be the rows except ('class'), while the Y data will simply be the column 'class'.

Below, we define functions to build 2 different kinds of simple neural networks:
1. A binary classification network: 
    - Takes matchup vectors (team_1 - team_2) as inputs, and outputs a binary digit prediction (1 if team_1 wins, 0 if team_2 wins).
2. A regression network: 
    - Takes matchup vectors (team_1 - team_2) as inputs, and outputs a scalar prediction for the difference between scores (team_1_score - team_2_score).
    - We then use these predicted scores to predict the matchup victor (1 if predicted score > 0, and 0 otherwise)
    
In order to evaluate the regression network's predictions, we define a prediction_percentage function which returns the percentage of games in which the victor is predicted correctly.

In [18]:
def pred_percentage(y_true, y_pred):
    y_true_sign = tf.sign(y_true)
    y_pred_sign = tf.sign(y_pred)
    mults = tf.multiply(y_true_sign, y_pred_sign)
    return K.mean(mults+1)/2

def build_keras_model(input_shape, num_units, num_layers, classification = True):
    model = Sequential()
    
    model.add(Dense(units = num_units, 
                    input_dim = input_shape[1], 
                    activation = 'relu')
                    kernel_regularizer = l1_l2(0, 0.001))
    
    model.add(Dropout(0.25))
    
    for i in range(num_layers - 1):
        model.add(Dense(num_units, 
                        activation = 'relu'
                        kernel_regularizer = l1_l2(0, 0.001)))
        model.add(Dropout(0.25))
    
    if classification:
        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(loss = 'binary_crossentropy', optimizer = Adam(), metrics = ['accuracy'])
    else:
        model.add(Dense(1))
        model.compile(loss = 'mae', optimizer = Adam(), metrics = [pred_percentage])
    return model

def train_and_evaluate(model, X_train, Y_train, X_test, Y_test, epochs):
    model.fit(X_train, Y_train, epochs = epochs, batch_size = 1)
    loss = model.evaluate(X_test, Y_test, batch_size = 1)
    print("Loss was: {} and accuracy was: {}".format(loss[0], loss[1]))
    return loss[1]

SyntaxError: invalid syntax (<ipython-input-18-655cc03774c2>, line 13)

Below, we use cross-validation (with 10 folds) to test the performance of our binary classification neural network. This process splits the data into 10 randomized chunks. Then, it uses $\frac{9}{10}$ chunks to train the network, while using the final chunk to test the predictions. After this, the average accuracy is printed below, and serves as a proxy for the performance of this neural network on unknown data.

In [16]:
X_train_class = train_matchup_data_class.drop(['class'], axis = 1)
input_shape = X_train_class.shape
Y_train_class = train_matchup_data_class['class']

num_units = 15
num_layers = 3

n_folds = 10
epochs = 5
folds = StratifiedKFold(Y_train_class, n_folds = n_folds, shuffle = True)


accuracies = []
for i, (train, test) in enumerate(folds):
    print("Running fold: {}".format(i+1))
    model = None
    model = build_keras_model(input_shape, num_units, num_layers, classification = True)
    accuracy = train_and_evaluate(model, X_train_class.iloc[train], 
                       Y_train_class.iloc[train], 
                       X_train_class.iloc[test], 
                       Y_train_class.iloc[test], 
                       epochs = epochs)
    accuracies.append(accuracy)

print("The average accuracy of predictions was: {}".format(np.mean(accuracies)))

Running fold: 1
Epoch 1/5
1282/1282 [==============================] - 4s 3ms/step - loss: 0.7284 - acc: 0.5577
Epoch 2/5
1282/1282 [==============================] - 4s 3ms/step - loss: 0.6677 - acc: 0.5835
Epoch 3/5
1282/1282 [==============================] - 3s 3ms/step - loss: 0.6386 - acc: 0.6154
Epoch 4/5
1282/1282 [==============================] - 3s 2ms/step - loss: 0.6324 - acc: 0.6443
Epoch 5/5
144/144 [==============================] - 0s 2ms/step
Loss was: 0.5916101426507036 and accuracy was: 0.6944444444444444
Running fold: 2
Epoch 1/5
1282/1282 [==============================] - 4s 3ms/step - loss: 0.7258 - acc: 0.5125
Epoch 2/5
1282/1282 [==============================] - 3s 2ms/step - loss: 0.6464 - acc: 0.6092
Epoch 3/5
1282/1282 [==============================] - 3s 2ms/step - loss: 0.6299 - acc: 0.6498A: 1s - 
Epoch 4/5
1282/1282 [==============================] - 3s 2ms/step - loss: 0.6015 - acc: 0.6739
Epoch 5/5
144/144 [==============================] - 0s 2ms/s

Now, we test the performance of our regression neural network. As above, we use cross validation (with 10 folds) in order to test our network on our given dataset. 

In [17]:
X_train_reg = train_matchup_data_reg.drop(['class'], axis = 1)
input_shape = X_train_reg.shape
Y_train_reg = train_matchup_data_reg['class']

num_units = 15
num_layers = 3

n_folds = 10
epochs = 5
folds = StratifiedKFold(Y_train_reg, n_folds = n_folds, shuffle = True)


accuracies = []
for i, (train, test) in enumerate(folds):
    print("Running fold: {}".format(i+1))
    model = None
    model = build_keras_model(input_shape, num_units, num_layers, classification = False)
    accuracy = train_and_evaluate(model, X_train_reg.iloc[train], 
                       Y_train_reg.iloc[train], 
                       X_train_reg.iloc[test], 
                       Y_train_reg.iloc[test], 
                       epochs = epochs)
    accuracies.append(accuracy)

print("The average accuracy of predictions was: {}".format(np.mean(accuracies)))

Running fold: 1


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Epoch 1/5
1262/1262 [==============================] - 4s 3ms/step - loss: 10.9941 - pred_percentage: 0.6101
Epoch 2/5
1262/1262 [==============================] - 3s 2ms/step - loss: 10.2886 - pred_percentage: 0.6767
Epoch 3/5
1262/1262 [==============================] - 3s 2ms/step - loss: 10.1585 - pred_percentage: 0.6878
Epoch 4/5
1262/1262 [==============================] - 3s 3ms/step - loss: 9.8238 - pred_percentage: 0.6957A: 0s - loss: 9.7443 - pr
Epoch 5/5
164/164 [==============================] - 1s 3ms/step
Loss was: 9.383550516715864 and accuracy was: 0.6829268292682927
Running fold: 2
Epoch 1/5
1265/1265 [==============================] - 4s 3ms/step - loss: 11.2359 - pred_percentage: 0.5462
Epoch 2/5
1265/1265 [==============================] - 3s 3ms/step - loss: 10.3326 - pred_percentage: 0.6553
Epoch 3/5
1265/1265 [==============================] - 3s 3ms/step - loss: 10.1564 - pred_percentage: 0.6632
Epoch 4/5
1265/1265 [==============================] - 3s 3ms/step 

The final neural network we will test is a bit more complex. Instead of taking a matchup vector as input, this neural network is simply provided the two team vectors. This allows the neural network to learn its own definition for a "matchup", instead of having to use ours. As with model #2, this is a regression network, which means its predictions represent the difference between the scores of the teams. 

In order to build this, we have to build our training set in a different way: building two different training sets in conjunction (one for team_1s and the other for team_2s), and then the labels in a third pandas series.

In [21]:
def gen_new_data(team_vector_store):
    """
    Generates data sets for inputs into the third neural network.
    """
    tourney_game_data = pd.read_csv("TourneyDetailedResults.csv")
    tourney_game_data = tourney_game_data[['Season', 'Wteam', 'Lteam', 'Wscore', 'Lscore']]

    # display(tourney_game_data)

    col_names = list(team_vector_store[2003])
    # print(list(col_names))

    team_ones = pd.DataFrame(columns = col_names)
    team_twos = pd.DataFrame(columns = col_names)

    one_minus_two = []
    
    for index, row in tourney_game_data.iterrows():
        season, wteam, lteam, wscore, lscore = row
        
        win_vector = team_vector_store[season].loc[wteam]
        lose_vector = team_vector_store[season].loc[lteam]
        
        #Win minus lose
        team_ones = team_ones.append(win_vector)
        team_twos = team_twos.append(lose_vector)
        one_minus_two.append(wscore-lscore)
    
        #Lose minus win
        team_ones = team_ones.append(lose_vector)
        team_twos = team_twos.append(win_vector)
        one_minus_two.append(lscore-wscore)
    
    one_minus_two = pd.Series(one_minus_two)
    return team_ones, team_twos, one_minus_two


In [22]:
new_team_ones, new_team_twos, one_minus_two = gen_new_data(team_vector_store)

Below, we define a function to create the third model.

In [23]:
def make_reg_model(team_feature_length):
    
    t1 = Input(shape = (team_feature_length, ))
    t2 = Input(shape = (team_feature_length, ))
    
    team_transform = Dense(team_feature_length, 
                            activation = 'tanh', 
                            kernel_regularizer = l1_l2(0, 0.001)
                          )
    
    team_1 = team_transform(t1)
    team_2 = team_transform(t2)
    
    merge = Concatenate(axis=-1)([team_1, team_2])
    
    dropout = Dropout(0.2)(merge)
    
    x = Dense(team_feature_length,
                activation = 'tanh',
                kernel_regularizer = l1_l2(0, 0.001)
             )(dropout)
    
    x = Dense(team_feature_length,
                activation = 'tanh',
                kernel_regularizer = l1_l2(0, 0.001)
             )(x)
    
    x = Dense(team_feature_length,
                activation = 'tanh',
                kernel_regularizer = l1_l2(0, 0.001)
             )(x)
    
    pred = Dense(1)(x) #linear activation for regression
    model = Model(inputs = [t1, t2],
                 outputs = pred)
    model.compile(optimizer = 'Adam', loss = 'mae', metrics = [pred_percentage])
    
    return model

def new_train_and_evaluate(model, X_train, Y_train, X_test, Y_test, epochs):
    model.fit(X_train, Y_train, epochs = epochs, batch_size = 1)
    loss = model.evaluate(X_test, Y_test, batch_size = 1)
    print("Loss was: {} and prediction percentage was: {}".format(loss[0], loss[1]))

Below, we test this third neural network using the same process as above.

In [24]:
# display(one_minus_two)
team_feature_length = len(imp_attributes)

n_folds = 10
epochs = 5
folds = StratifiedKFold(one_minus_two, n_folds = n_folds, shuffle = True)
# folds = StratifiedKFold(n_folds = n_folds, shuffle=True)

percentages = []
for i, (train, test) in enumerate(folds):
    print("Running fold: {}".format(i+1))
    model = None
    model = make_reg_model(team_feature_length)
    percentage =train_and_evaluate(model,
                       [new_team_ones.iloc[train], new_team_twos.iloc[train]],
                       one_minus_two.iloc[train], 
                       [new_team_ones.iloc[test], new_team_twos.iloc[test]],
                       one_minus_two.iloc[test], 
                       epochs = epochs)
    percentages.append(percentage)

print("Average prediction accuracy: {}".format(np.mean(percentages)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


Running fold: 1
Epoch 1/5
1270/1270 [==============================] - 4s 3ms/step - loss: 10.4591 - pred_percentage: 0.6843
Epoch 2/5
1270/1270 [==============================] - 3s 2ms/step - loss: 9.7916 - pred_percentage: 0.7181
Epoch 3/5
1270/1270 [==============================] - 3s 2ms/step - loss: 9.6639 - pred_percentage: 0.7087
Epoch 4/5
1270/1270 [==============================] - 3s 2ms/step - loss: 9.4346 - pred_percentage: 0.7252
Epoch 5/5
156/156 [==============================] - 1s 4ms/step
Loss was: 8.734520721320923 and accuracy was: 0.7051282051282052
Running fold: 2
Epoch 1/5
1266/1266 [==============================] - 5s 4ms/step - loss: 10.5428 - pred_percentage: 0.6596
Epoch 2/5
1266/1266 [==============================] - 4s 3ms/step - loss: 9.7503 - pred_percentage: 0.6998
Epoch 3/5
1266/1266 [==============================] - 3s 3ms/step - loss: 9.4108 - pred_percentage: 0.7180A: 0s - loss: 9.4242 - pred_percentag
Epoch 4/5
1266/1266 [======================

We have now created and tested 3 different models:
1. Simple binary classifier. 
    - Uses our definition of matchups (team_1 - team_2), and outputs 1 if team_1 wins, 0 if team_2 wins.

2. Simple Regression.
    - Uses our definition of matchups, and tries to predict the difference between points of team_1 and team_2.
    - We then use a scoring function (return 1 if score_diff > 0, and 0 otherwise) in order to generate predictions.

3. More Complex Regression.
    - We feed just the team vectors into the model, and let the model learn an effective definition for what a matchup is.
    - The model then outputs a prediction of the score differential (team_1_score - team_2_score).
    - We then use the scoring function defined above to generate predictions.
    
    

Now we will use these models (and the training data) in order to generate predictions for the matchups between 2011-2013.

In [25]:
def generate_matchup_testing_data(team_vector_store):
    """
    Function to generate the training/testing data.
    Based on team_vector_store and the tourney detailed results.
    If classification = True, generates for binary classification.
    If classification = False, generates for regression.
    
    Returns training_data
    """
    col_names = list(team_vector_store[2003])

    testing_matchup_data = pd.DataFrame(columns = col_names)
    matchups = []
    
    for year in [2011, 2012, 2013]:
        season_data = team_vector_store[year].sort_index()
        for i1, row1 in season_data.iterrows():
            for i2, row2 in season_data.iterrows():
#                 print(row2)
                if i2 > i1:
                    matchups.append([year, i1, i2])
                    matchup = row1 - row2
                    testing_matchup_data = testing_matchup_data.append([matchup], ignore_index = True)

    return testing_matchup_data, matchups

In [26]:
matchup_data, matchups = generate_matchup_testing_data(team_vector_store)

In [27]:
def generate_predictions_to_csv(model, X_train, Y_train, outfile, testing_data, matchups, classification = True):
    model.fit(X_train, Y_train, epochs = 5, batch_size = 1)
    if classification:
        predictions = model.predict_classes(testing_data)
    else:
        predictions = model.predict(testing_data)
        
    result = []
    for i in range(len(matchups)):
        representation = '_'.join([str(matchups[i][0]), str(matchups[i][1]), str(matchups[i][2])])
        if classification:
            representation = [representation] + [str(predictions[i][0])]
        else:
            pred = 1 if predictions[i][0] > 0 else 0
            representation = [representation] + [str(pred)]
        result.append(representation)

    # print(result)
    result = pd.DataFrame(data = result, columns = ['game_ID', 'Prediction'])
    result.to_csv(outfile, index = False, sep = ',')

In [28]:
X_train_class = train_matchup_data_class.drop(['class'], axis = 1)
input_shape = X_train_class.shape
Y_train_class = train_matchup_data_class['class']
binary_class_model = build_keras_model(input_shape, 
                                       num_units = 15, 
                                       num_layers = 3, 
                                       classification = True)

generate_predictions_to_csv(binary_class_model, 
                            X_train_class, 
                            Y_train_class, 
                            "bin_predictions.csv",
                            matchup_data,
                            matchups,
                            classification = True)

Epoch 1/5
1426/1426 [==============================] - 5s 4ms/step - loss: 0.7464 - acc: 0.5554
Epoch 2/5
1426/1426 [==============================] - 3s 2ms/step - loss: 0.6533 - acc: 0.6094
Epoch 3/5
1426/1426 [==============================] - 3s 2ms/step - loss: 0.6179 - acc: 0.6711
Epoch 4/5
1426/1426 [==============================] - 3s 2ms/step - loss: 0.6074 - acc: 0.6795
Epoch 5/5
1426/1426 [==============================] - 3s 2ms/step - loss: 0.5976 - acc: 0.6935A: 0s - loss: 0.


In [29]:
X_train_reg = train_matchup_data_reg.drop(['class'], axis = 1)
input_shape = X_train_reg.shape
Y_train_reg = train_matchup_data_reg['class']

simple_regression_model = build_keras_model(input_shape, 
                                            num_units = 15, 
                                            num_layers = 3, 
                                            classification = False)

generate_predictions_to_csv(simple_regression_model, 
                            X_train_reg, 
                            Y_train_reg, 
                            "simple_reg_predictions.csv",
                            matchup_data,
                            matchups,
                            classification = False)

Epoch 1/5
1426/1426 [==============================] - 5s 3ms/step - loss: 11.2351 - pred_percentage: 0.5631
Epoch 2/5
1426/1426 [==============================] - 3s 2ms/step - loss: 10.1438 - pred_percentage: 0.6648
Epoch 3/5
1426/1426 [==============================] - 3s 2ms/step - loss: 10.0996 - pred_percentage: 0.6732
Epoch 4/5
1426/1426 [==============================] - 3s 2ms/step - loss: 9.9595 - pred_percentage: 0.6985
Epoch 5/5
1426/1426 [==============================] - 3s 2ms/step - loss: 9.6019 - pred_percentage: 0.7069


In [30]:
def generate_indiv_testing_data(team_vector_store):
    """
    Function to generate the testing data.
    """
    col_names = list(team_vector_store[2003])

    team_ones = pd.DataFrame(columns = col_names)
    team_twos = pd.DataFrame(columns = col_names)
    matchups = []
    
    for year in [2011, 2012, 2013]:
        season_data = team_vector_store[year].sort_index()
        for i1, row1 in season_data.iterrows():
            for i2, row2 in season_data.iterrows():
#                 print(row2)
                if i2 > i1:
                    team_ones = team_ones.append(row1, ignore_index = True)
                    team_twos = team_twos.append(row2, ignore_index = True)
                    matchups.append([year, i1, i2])
                    
    return [team_ones, team_twos], matchups

In [31]:
X_train = [new_team_ones, new_team_twos]
team_dim = len(imp_attributes)
Y_train = one_minus_two

complex_regression_model = make_reg_model(team_dim)
indiv_testing_data, matchups = generate_indiv_testing_data(team_vector_store)

generate_predictions_to_csv(complex_regression_model,
                           X_train,
                           Y_train,
                           "complex_reg_predictions.csv",
                           indiv_testing_data,
                           matchups,
                           classification = False)

Epoch 1/5
1426/1426 [==============================] - 6s 4ms/step - loss: 10.4085 - pred_percentage: 0.6732
Epoch 2/5
1426/1426 [==============================] - 4s 3ms/step - loss: 9.6723 - pred_percentage: 0.7062
Epoch 3/5
1426/1426 [==============================] - 4s 3ms/step - loss: 9.4016 - pred_percentage: 0.7160
Epoch 4/5
1426/1426 [==============================] - 4s 3ms/step - loss: 9.3995 - pred_percentage: 0.7153
Epoch 5/5
1426/1426 [==============================] - 4s 3ms/step - loss: 9.2462 - pred_percentage: 0.7090
